# CAM application
Notebook developped on colab (for various techniqcal reasons) to run CAM-based techniques on the dataset loaded on drive.

In [ ]:
# link colab and drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
# then follow passages

Mounted at /content/drive


# Start

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from PIL import Image
import os

import torch
from torch import nn
import timm
from pytorch_grad_cam import GradCAM, GradCAMPlusPlus, ScoreCAM, HiResCAM,  GradCAMElementWise, XGradCAM, LayerCAM, EigenCAM, AblationCAM, FullGrad, EigenGradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image

In [ ]:
# cuda or cpu (choose flag)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))

Device: cuda:0


In [ ]:
# # mean and standard deviation of the train dataset
# MEAN = [0.3895, 0.4890, 0.4233]
# STD = [0.2877, 0.2396, 0.2902]

# IMAGE_SIZE = 768

In [ ]:
# utility function to display heatmap
def make_image(data, outputname, size=(20, 16), dpi=64):
    fig = plt.figure()
    fig.set_size_inches(size)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.set_cmap('hot')
    ax.imshow(data, aspect='equal')
    plt.savefig(outputname, dpi=dpi)
    plt.close()
    plt.clf()

In [ ]:
# modification of the library to avoid the preprocess of the image
from torchvision.transforms import Compose, Normalize, ToTensor

def preprocess_image_2(
    img: np.ndarray, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) -> torch.Tensor:
    preprocessing = Compose([
            ToTensor(),
            # Normalize(mean=mean, std=std)
            ])
    return preprocessing(img.copy()).unsqueeze(0)

In [ ]:
maps = ['AblationCAM',
 'EigenCAM',
 'EigenGradCAM',
 'FullGrad',
 'GradCAM',
 'GradCAMElementWise',
 'GradCAMPlusPlus',
 'HiResCAM',
 'LayerCAM',
 'XGradCAM'
 ]

# CAM

In [ ]:
# Central function used to easily apply all the CAM-based techniques

def one_only(device, path, where_model, version, cam_type):
    # load model...

    model = timm.create_model('resnet50d', pretrained=True)

    model.fc = nn.Sequential(
        nn.Linear(2048, 512),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(512, 2),
    )

    model = model.to(device)

    # load its weights
    final = torch.load(where_model,  map_location='cuda:0')
    model.load_state_dict(final['model_state_dict'])

    model = model.eval()

    for filename in tqdm(os.listdir(path)):

        # image setting
        pp = path+filename
        rgb_img = np.array(Image.open(str(pp)))
        rgb_img = np.float32(rgb_img) / 255
        input_tensor = preprocess_image_2(rgb_img)#, mean=MEAN, std=STD)

        # model + image
        input_tensor = input_tensor.to(device)

        output = model(input_tensor)
        index = (torch.max(output, 1)).indices.item()

        # print(output.softmax(dim=1), ' --- ', index )

        # layers = [model.layer1[-1], model.layer2[-1], model.layer3[-1], model.layer4[-1]] # <-- modify if need different names
        layers = [model.layer4[-1]]
        name = 0

        for layer in layers:

            name += 1

            if cam_type == 'AblationCAM':
              cam = AblationCAM(model = model, target_layers=[layer], use_cuda=True)
            if cam_type == 'EigenCAM':
              cam = EigenCAM(model = model, target_layers=[layer], use_cuda=True)
            if cam_type == 'EigenGradCAM':
              cam = EigenGradCAM(model = model, target_layers=[layer], use_cuda=True)
            if cam_type == 'FullGrad':
              cam = FullGrad(model = model, target_layers=[layer], use_cuda=True)
            if cam_type == 'GradCAM':
              cam = GradCAM(model = model, target_layers=[layer], use_cuda=True)
            if cam_type == 'GradCAMElementWise':
              cam = GradCAMElementWise(model = model, target_layers=[layer], use_cuda=True)
            if cam_type == 'GradCAMPlusPlus':
              cam = GradCAMPlusPlus(model = model, target_layers=[layer], use_cuda=True)
            if cam_type == 'HiResCAM':
              cam = HiResCAM(model = model, target_layers=[layer], use_cuda=True)
            if cam_type == 'LayerCAM':
              cam = LayerCAM(model = model, target_layers=[layer], use_cuda=True)
            if cam_type == 'XGradCAM':
              cam = XGradCAM(model = model, target_layers=[layer], use_cuda=True)

            grayscale_cam = cam(input_tensor=input_tensor)#, aug_smooth=True, eigen_smooth=True)
            grayscale_cam = grayscale_cam[0, :]

            visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True, image_weight = 0) # param image_weight: The final result is image_weight * img + (1-image_weight) * mask


            os.makedirs(f'/content/drive/MyDrive/Oliogel/cam_res/{cam_type}_{version}/{name}', exist_ok=True)
            os.makedirs(f'/content/drive/MyDrive/Oliogel/cam_res/{cam_type}/D_cam', exist_ok=True)

            where_gc = os.path.join(f'/content/drive/MyDrive/Oliogel/cam_res/{cam_type}_{version}/{name}', f'{filename}___{index}'.replace('.tif',''))
            where_cam = os.path.join(f'/content/drive/MyDrive/Oliogel/cam_res/{cam_type}/D_cam/{layer}', f'{filename}___{index}'.replace('.tif',''))

            np.savetxt(f"{where_cam}.csv", grayscale_cam, delimiter=",")
            make_image(visualization, f'{where_gc}.png')

In [ ]:
one_only(device = device,
         path = '/content/drive/MyDrive/Oliogel/TEST_5/test_set/',
         where_model = '/content/drive/MyDrive/Oliogel/model/model.pth',
         version = '5',
         cam_type = 'LayerCAM')

100%|██████████| 24/24 [00:22<00:00,  1.09it/s]


<Figure size 640x480 with 0 Axes>